In [2]:
import pandas as pd
import numpy as np
import json
import os
import unidecode
import unicodedata as ud
import regex as re

In [2]:
def read_json(data_dir, filename):
    with open(f'{data_dir}/{filename}', 'r') as openfile:
        return json.load(openfile)

In [3]:
def map_industry_index_to_text(df, inds_df):
    mapped_industries_ls = []
    for row in df.industry_index:
        inds_ls = row.split(', ')
        mapped_ls = []
        for ind in inds_ls:
            mapped_ind = inds_df.loc[inds_df['index']==int(ind)]['name'].values[0]
            mapped_ls.append(mapped_ind)
        mapped_industries_ls.append(', '.join(mapped_ls))
    return mapped_industries_ls

In [4]:
full_inds_df = pd.read_csv('data/topcv/csv/mapping-titles - topcv_industries.csv', encoding='utf-8')

In [4]:
industries = read_json('data/topcv/json','industries_raw.json')

In [2]:
topcv_r = pd.read_csv("data/topcv/csv/topcv_raw.csv")
topcv_r

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr
0,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Tư Vấn,Hạn nộp hồ sơ: 31/05/2023,CÔNG TY TÀI CHÍNH TNHH MTV NGÂN HÀNG TMCP SÀI ...,Trên 10 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Đà Nẵng: 324 NGUYỄN HỮU THỌ,KHUÊ TRUNG,CẨM...","<div class=""content-tab""><p>- Tư vấn giới thiệ...","<div class=""content-tab""><p>- Tuổi từ 20 trở l...",Nhân viên,0,"Đà Nẵng, Thừa Thiên Huế, Quảng Nam"
1,https://www.topcv.vn/viec-lam/telesales-mang-b...,Telesales Mảng Bất Động Sản (Thu Nhập Hấp Dẫn ...,Hạn nộp hồ sơ: 08/06/2023,Công ty TNHH Alliga Eternity,15 - 20 triệu,Toàn thời gian,1 năm,"['- Hà Nội: Tầng 7, Toà nhà VIB, 59 Quang Trun...","<div class=""content-tab""><ul><li>Liên hệ đặt l...","<div class=""content-tab""><ul><li>Nam/nữ từ 22 ...",Nhân viên,0,Hà Nội
2,https://www.topcv.vn/viec-lam/post-catalogue-m...,Post Catalogue Manager/ Supervisor ( Marketing...,Hạn nộp hồ sơ: 24/05/2023,Lotte mart,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: 469 Nguyễn Hữu Thọ, Phường Tâ...","<div class=""content-tab""><p>Responsibility 1: ...","<div class=""content-tab""><p>Education: BA degr...",Quản lý / Giám sát,0,Hồ Chí Minh
3,https://www.topcv.vn/viec-lam/nhan-vien-ban-ha...,Nhân Viên Bán Hàng Tại Viettelstore Tiểu Khu T...,Hạn nộp hồ sơ: 22/05/2023,Hệ thống Bán lẻ Viettel store - Công ty TM & X...,8 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,['- Hà Nội: Phú Xuyên'],"<div class=""content-tab""><p>+ Bán hàng và chăm...","<div class=""content-tab""><p>+ Nữ, tuổi từ 20-2...",Nhân viên,0,Hà Nội
4,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh / Sales,Hạn nộp hồ sơ: 30/06/2023,Công ty TNHH Sản xuất và Thương mại Công nghiệ...,Trên 10 triệu,Toàn thời gian,1 năm,"['- Hà Nội: Tòa nhà Golden Palm, số 21 Lê Văn ...","<div class=""content-tab""><ul><li>Nắm rõ sản ph...","<div class=""content-tab""><ul><li>Tốt nghiệp Ca...",Nhân viên,0,Hà Nội
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75903,https://www.topcv.vn/viec-lam/business-analyst...,Business Analyst (IT Ba),Hạn nộp hồ sơ: 21/06/2023,Tập Đoàn Novaon,12 - 18 triệu,Toàn thời gian,2 năm,['- Hà Nội: Cầu Giấy'],"<div class=""content-tab""><p>● Tiếp nhận, khảo ...","<div class=""content-tab""><p>● Có từ 2 năm kinh...",Nhân viên,66,Hà Nội
75904,https://www.topcv.vn/viec-lam/product-designer...,Product Designer,Hạn nộp hồ sơ: 18/06/2023,Tập Đoàn Novaon,15 - 27 triệu,Toàn thời gian,2 năm,"['- Hà Nội: Tầng 3, tòa nhà CIC, số 219 Trung ...","<div class=""content-tab""><ul><li>Nghiên cứu, p...","<div class=""content-tab""><ul><li>Có kỹ năng, h...",Nhân viên,66,Hà Nội
75905,https://www.topcv.vn/viec-lam/senior-account-m...,Senior Account Manager,Hạn nộp hồ sơ: 21/05/2023,Công ty TNHH giải pháp chuyển đổi số Việt Nam,Tới 25 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: 116 Trung Liệt, Đống Đa']","<div class=""content-tab""><p>• Thực hiện việc f...","<div class=""content-tab""><p>• Tốt nghiệp Cao đ...",Trưởng nhóm,66,Hà Nội
75906,https://www.topcv.vn/viec-lam/chuyen-vien-kinh...,Chuyên Viên Kinh Doanh Phần Cứng Kiotviet - Tâ...,Hạn nộp hồ sơ: 28/05/2023,CÔNG TY CỔ PHẦN CÔNG NGHỆ KIOTVIET,Tới 15 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,['- Hồ Chí Minh: Lầu 6 - Khu C. Tòa nhà WASECO...,"<div class=""content-tab""><p>-Liên hệ với khách...","<div class=""content-tab""><p>-Trình độ trung cấ...",Nhân viên,66,Hồ Chí Minh


### Attributes needed

- **date** : date posted
- **title** : job title
- **industry** : example: Information Technology, Healthcare, Accounting, etc.
- **level** : ex: employee, manager, etc.
- **salary** : min - max
- **experience** : years of experience (min - max)
- **city** : cities / provinces of Vietnam
- **description** : job responsibilities, requirements, skills needed, etc.

### 1. Processing date attribute

In [11]:
topcv_r['modified_deadline'] = pd.to_datetime(pd.Series([job.split(':')[-1].strip() for job in topcv_r['job_deadline']]), format='%d/%m/%Y')

In [13]:
topcv_r.head(5)

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr,modified_deadline
0,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Tư Vấn,Hạn nộp hồ sơ: 31/05/2023,CÔNG TY TÀI CHÍNH TNHH MTV NGÂN HÀNG TMCP SÀI ...,Trên 10 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Đà Nẵng: 324 NGUYỄN HỮU THỌ,KHUÊ TRUNG,CẨM...","<div class=""content-tab""><p>- Tư vấn giới thiệ...","<div class=""content-tab""><p>- Tuổi từ 20 trở l...",Nhân viên,0,"Đà Nẵng, Thừa Thiên Huế, Quảng Nam",2023-05-31
1,https://www.topcv.vn/viec-lam/telesales-mang-b...,Telesales Mảng Bất Động Sản (Thu Nhập Hấp Dẫn ...,Hạn nộp hồ sơ: 08/06/2023,Công ty TNHH Alliga Eternity,15 - 20 triệu,Toàn thời gian,1 năm,"['- Hà Nội: Tầng 7, Toà nhà VIB, 59 Quang Trun...","<div class=""content-tab""><ul><li>Liên hệ đặt l...","<div class=""content-tab""><ul><li>Nam/nữ từ 22 ...",Nhân viên,0,Hà Nội,2023-06-08
2,https://www.topcv.vn/viec-lam/post-catalogue-m...,Post Catalogue Manager/ Supervisor ( Marketing...,Hạn nộp hồ sơ: 24/05/2023,Lotte mart,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: 469 Nguyễn Hữu Thọ, Phường Tâ...","<div class=""content-tab""><p>Responsibility 1: ...","<div class=""content-tab""><p>Education: BA degr...",Quản lý / Giám sát,0,Hồ Chí Minh,2023-05-24
3,https://www.topcv.vn/viec-lam/nhan-vien-ban-ha...,Nhân Viên Bán Hàng Tại Viettelstore Tiểu Khu T...,Hạn nộp hồ sơ: 22/05/2023,Hệ thống Bán lẻ Viettel store - Công ty TM & X...,8 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,['- Hà Nội: Phú Xuyên'],"<div class=""content-tab""><p>+ Bán hàng và chăm...","<div class=""content-tab""><p>+ Nữ, tuổi từ 20-2...",Nhân viên,0,Hà Nội,2023-05-22
4,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh / Sales,Hạn nộp hồ sơ: 30/06/2023,Công ty TNHH Sản xuất và Thương mại Công nghiệ...,Trên 10 triệu,Toàn thời gian,1 năm,"['- Hà Nội: Tòa nhà Golden Palm, số 21 Lê Văn ...","<div class=""content-tab""><ul><li>Nắm rõ sản ph...","<div class=""content-tab""><ul><li>Tốt nghiệp Ca...",Nhân viên,0,Hà Nội,2023-06-30


### 2. Mapping industry

Detecting the same jobs that appear in different industries

In [36]:
url_value_counts = topcv_r['url'].value_counts()

In [37]:
url_value_counts.index[0]

'https://www.topcv.vn/viec-lam/chuyen-vien-cao-cap-tu-van-tai-chinh-mang-tin-dung-khach-hang-ca-nhan-chi-nhanh-nam-tu-liem-ha-noi/894295.html?ta_source=JobSearchList'

In [53]:
topcv_r[topcv_r['url'] == url_value_counts.index[1]]

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr,modified_deadline
12596,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Hạn nộp hồ sơ: 15/06/2023,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: Tòa nhà PaxSky, 26 Ung Văn Kh...","<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,1,Hồ Chí Minh,2023-06-15
15052,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Hạn nộp hồ sơ: 15/06/2023,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: Tòa nhà PaxSky, 26 Ung Văn Kh...","<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,4,Hồ Chí Minh,2023-06-15
73531,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Hạn nộp hồ sơ: 15/06/2023,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: Tòa nhà PaxSky, 26 Ung Văn Kh...","<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,66,Hồ Chí Minh,2023-06-15


In [70]:
dict(topcv_r[topcv_r['url'] == url_value_counts.index[1]].iloc[0])['industry_index'].astype(str)

'1'

In [73]:
unique_jobs_ls = []
count = 0
for url in url_value_counts.index:
    inds = []
    unique_job = dict(topcv_r[topcv_r['url'] == url].iloc[0])
    unique_job['industry_index'] = unique_job['industry_index'].astype(str)
    for ind,row in topcv_r[topcv_r['url'] == url].iterrows():
        if ind == 0: continue
        temp_ind = row['industry_index']
        unique_job['industry_index'] += f', {temp_ind}'
    print(f"Finished url {count}")
    count += 1
    unique_jobs_ls.append(unique_job)

Finished url 0
Finished url 1
Finished url 2
Finished url 3
Finished url 4
Finished url 5
Finished url 6
Finished url 7
Finished url 8
Finished url 9
Finished url 10
Finished url 11
Finished url 12
Finished url 13
Finished url 14
Finished url 15
Finished url 16
Finished url 17
Finished url 18
Finished url 19
Finished url 20
Finished url 21
Finished url 22
Finished url 23
Finished url 24
Finished url 25
Finished url 26
Finished url 27
Finished url 28
Finished url 29
Finished url 30
Finished url 31
Finished url 32
Finished url 33
Finished url 34
Finished url 35
Finished url 36
Finished url 37
Finished url 38
Finished url 39
Finished url 40
Finished url 41
Finished url 42
Finished url 43
Finished url 44
Finished url 45
Finished url 46
Finished url 47
Finished url 48
Finished url 49
Finished url 50
Finished url 51
Finished url 52
Finished url 53
Finished url 54
Finished url 55
Finished url 56
Finished url 57
Finished url 58
Finished url 59
Finished url 60
Finished url 61
Finished url 62
Fi

Finished url 511
Finished url 512
Finished url 513
Finished url 514
Finished url 515
Finished url 516
Finished url 517
Finished url 518
Finished url 519
Finished url 520
Finished url 521
Finished url 522
Finished url 523
Finished url 524
Finished url 525
Finished url 526
Finished url 527
Finished url 528
Finished url 529
Finished url 530
Finished url 531
Finished url 532
Finished url 533
Finished url 534
Finished url 535
Finished url 536
Finished url 537
Finished url 538
Finished url 539
Finished url 540
Finished url 541
Finished url 542
Finished url 543
Finished url 544
Finished url 545
Finished url 546
Finished url 547
Finished url 548
Finished url 549
Finished url 550
Finished url 551
Finished url 552
Finished url 553
Finished url 554
Finished url 555
Finished url 556
Finished url 557
Finished url 558
Finished url 559
Finished url 560
Finished url 561
Finished url 562
Finished url 563
Finished url 564
Finished url 565
Finished url 566
Finished url 567
Finished url 568
Finished url 5

Finished url 1001
Finished url 1002
Finished url 1003
Finished url 1004
Finished url 1005
Finished url 1006
Finished url 1007
Finished url 1008
Finished url 1009
Finished url 1010
Finished url 1011
Finished url 1012
Finished url 1013
Finished url 1014
Finished url 1015
Finished url 1016
Finished url 1017
Finished url 1018
Finished url 1019
Finished url 1020
Finished url 1021
Finished url 1022
Finished url 1023
Finished url 1024
Finished url 1025
Finished url 1026
Finished url 1027
Finished url 1028
Finished url 1029
Finished url 1030
Finished url 1031
Finished url 1032
Finished url 1033
Finished url 1034
Finished url 1035
Finished url 1036
Finished url 1037
Finished url 1038
Finished url 1039
Finished url 1040
Finished url 1041
Finished url 1042
Finished url 1043
Finished url 1044
Finished url 1045
Finished url 1046
Finished url 1047
Finished url 1048
Finished url 1049
Finished url 1050
Finished url 1051
Finished url 1052
Finished url 1053
Finished url 1054
Finished url 1055
Finished u

Finished url 1460
Finished url 1461
Finished url 1462
Finished url 1463
Finished url 1464
Finished url 1465
Finished url 1466
Finished url 1467
Finished url 1468
Finished url 1469
Finished url 1470
Finished url 1471
Finished url 1472
Finished url 1473
Finished url 1474
Finished url 1475
Finished url 1476
Finished url 1477
Finished url 1478
Finished url 1479
Finished url 1480
Finished url 1481
Finished url 1482
Finished url 1483
Finished url 1484
Finished url 1485
Finished url 1486
Finished url 1487
Finished url 1488
Finished url 1489
Finished url 1490
Finished url 1491
Finished url 1492
Finished url 1493
Finished url 1494
Finished url 1495
Finished url 1496
Finished url 1497
Finished url 1498
Finished url 1499
Finished url 1500
Finished url 1501
Finished url 1502
Finished url 1503
Finished url 1504
Finished url 1505
Finished url 1506
Finished url 1507
Finished url 1508
Finished url 1509
Finished url 1510
Finished url 1511
Finished url 1512
Finished url 1513
Finished url 1514
Finished u

Finished url 1924
Finished url 1925
Finished url 1926
Finished url 1927
Finished url 1928
Finished url 1929
Finished url 1930
Finished url 1931
Finished url 1932
Finished url 1933
Finished url 1934
Finished url 1935
Finished url 1936
Finished url 1937
Finished url 1938
Finished url 1939
Finished url 1940
Finished url 1941
Finished url 1942
Finished url 1943
Finished url 1944
Finished url 1945
Finished url 1946
Finished url 1947
Finished url 1948
Finished url 1949
Finished url 1950
Finished url 1951
Finished url 1952
Finished url 1953
Finished url 1954
Finished url 1955
Finished url 1956
Finished url 1957
Finished url 1958
Finished url 1959
Finished url 1960
Finished url 1961
Finished url 1962
Finished url 1963
Finished url 1964
Finished url 1965
Finished url 1966
Finished url 1967
Finished url 1968
Finished url 1969
Finished url 1970
Finished url 1971
Finished url 1972
Finished url 1973
Finished url 1974
Finished url 1975
Finished url 1976
Finished url 1977
Finished url 1978
Finished u

...

Finished url 28759
Finished url 28760
Finished url 28761
Finished url 28762
Finished url 28763
Finished url 28764
Finished url 28765
Finished url 28766
Finished url 28767
Finished url 28768
Finished url 28769
Finished url 28770
Finished url 28771
Finished url 28772
Finished url 28773
Finished url 28774
Finished url 28775
Finished url 28776
Finished url 28777
Finished url 28778
Finished url 28779
Finished url 28780
Finished url 28781
Finished url 28782
Finished url 28783
Finished url 28784
Finished url 28785
Finished url 28786
Finished url 28787
Finished url 28788
Finished url 28789
Finished url 28790
Finished url 28791
Finished url 28792
Finished url 28793
Finished url 28794
Finished url 28795
Finished url 28796
Finished url 28797
Finished url 28798
Finished url 28799
Finished url 28800
Finished url 28801
Finished url 28802
Finished url 28803
Finished url 28804
Finished url 28805
Finished url 28806
Finished url 28807
Finished url 28808
Finished url 28809
Finished url 28810
Finished url

Finished url 29192
Finished url 29193
Finished url 29194
Finished url 29195
Finished url 29196
Finished url 29197
Finished url 29198
Finished url 29199
Finished url 29200
Finished url 29201
Finished url 29202
Finished url 29203
Finished url 29204
Finished url 29205
Finished url 29206
Finished url 29207
Finished url 29208
Finished url 29209
Finished url 29210
Finished url 29211
Finished url 29212
Finished url 29213
Finished url 29214
Finished url 29215
Finished url 29216
Finished url 29217
Finished url 29218
Finished url 29219
Finished url 29220
Finished url 29221
Finished url 29222
Finished url 29223
Finished url 29224
Finished url 29225
Finished url 29226
Finished url 29227
Finished url 29228
Finished url 29229
Finished url 29230
Finished url 29231
Finished url 29232
Finished url 29233
Finished url 29234
Finished url 29235
Finished url 29236
Finished url 29237
Finished url 29238
Finished url 29239
Finished url 29240
Finished url 29241
Finished url 29242
Finished url 29243
Finished url

Finished url 29635
Finished url 29636
Finished url 29637
Finished url 29638
Finished url 29639
Finished url 29640
Finished url 29641
Finished url 29642
Finished url 29643
Finished url 29644
Finished url 29645
Finished url 29646
Finished url 29647
Finished url 29648
Finished url 29649
Finished url 29650
Finished url 29651
Finished url 29652
Finished url 29653
Finished url 29654
Finished url 29655
Finished url 29656
Finished url 29657
Finished url 29658
Finished url 29659
Finished url 29660
Finished url 29661
Finished url 29662
Finished url 29663
Finished url 29664
Finished url 29665
Finished url 29666
Finished url 29667
Finished url 29668
Finished url 29669
Finished url 29670
Finished url 29671
Finished url 29672
Finished url 29673
Finished url 29674
Finished url 29675
Finished url 29676
Finished url 29677
Finished url 29678
Finished url 29679
Finished url 29680
Finished url 29681
Finished url 29682
Finished url 29683
Finished url 29684
Finished url 29685
Finished url 29686
Finished url

Finished url 30079
Finished url 30080
Finished url 30081
Finished url 30082
Finished url 30083
Finished url 30084
Finished url 30085
Finished url 30086
Finished url 30087
Finished url 30088
Finished url 30089
Finished url 30090
Finished url 30091
Finished url 30092
Finished url 30093
Finished url 30094
Finished url 30095
Finished url 30096
Finished url 30097
Finished url 30098
Finished url 30099
Finished url 30100
Finished url 30101
Finished url 30102
Finished url 30103
Finished url 30104
Finished url 30105
Finished url 30106
Finished url 30107
Finished url 30108
Finished url 30109
Finished url 30110
Finished url 30111
Finished url 30112
Finished url 30113
Finished url 30114
Finished url 30115
Finished url 30116
Finished url 30117
Finished url 30118
Finished url 30119
Finished url 30120
Finished url 30121
Finished url 30122
Finished url 30123
Finished url 30124
Finished url 30125
Finished url 30126
Finished url 30127
Finished url 30128
Finished url 30129
Finished url 30130
Finished url

Finished url 30528
Finished url 30529
Finished url 30530
Finished url 30531
Finished url 30532
Finished url 30533
Finished url 30534
Finished url 30535
Finished url 30536
Finished url 30537
Finished url 30538
Finished url 30539
Finished url 30540
Finished url 30541


In [77]:
topcv_r2 = pd.DataFrame.from_records(unique_jobs_ls) 

In [102]:
topcv_r2

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr,modified_deadline
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Hạn nộp hồ sơ: 30/06/2023,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Shinhan Bank, Tầng 4, Tòa Goldenpa...","<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,"13, 13, 29, 33",Hà Nội,2023-06-30
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Hạn nộp hồ sơ: 15/06/2023,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: Tòa nhà PaxSky, 26 Ung Văn Kh...","<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,"1, 1, 4, 66",Hồ Chí Minh,2023-06-15
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",Hạn nộp hồ sơ: 09/06/2023,CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,['- Cần Thơ: Thốt Nốt'],"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,"11, 11, 14, 36",Cần Thơ,2023-06-09
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,Hạn nộp hồ sơ: 09/06/2023,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"['- Hà Nội: Số 19 phố nguyễn sơn, phường ngọc ...","<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,"11, 11, 52, 62",Hà Nội,2023-06-09
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Hạn nộp hồ sơ: 31/05/2023,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Trường Đại học Thành Đô, Hoài Đức']","<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,"19, 19, 23, 66",Hà Nội,2023-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,https://www.topcv.vn/viec-lam/nhan-vien-inboun...,Nhân Viên Inbound Sales,Hạn nộp hồ sơ: 30/06/2023,Công ty TNHH Đầu tư và Du lịch 365,Trên 9 triệu,Toàn thời gian,3 năm,"['- Hà Nội: Tầng 16, Toà nhà Ladeco, 266 Đội C...","<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...",Nhân viên,"25, 25","Hà Nội, Hồ Chí Minh",2023-06-30
30538,https://www.topcv.vn/viec-lam/dieu-duong-truon...,Điều Dưỡng Trưởng Bệnh Viện,Hạn nộp hồ sơ: 24/05/2023,Bệnh Viện Quốc Tế DoLife,Thoả thuận,Toàn thời gian,2 năm,"['- Hà Nội: 108 Nguyễn Hoàng, Mỹ Đình, Nam Từ ...","<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Trưởng/Phó phòng,"8, 8",Hà Nội,2023-05-24
30539,https://www.topcv.vn/viec-lam/nhan-vien-le-tan...,Nhân Viên Lễ Tân,Hạn nộp hồ sơ: 24/05/2023,Công ty Cổ phần Dịch vụ Y tế SBB,7 - 12 triệu,Toàn thời gian,Dưới 1 năm,"['- Hà Nội: 499 Trần Khát Chân, Hai Bà Trưng']","<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",Nhân viên,"8, 8",Hà Nội,2023-05-24
30540,https://www.topcv.vn/viec-lam/nhan-vien-dat-li...,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,Hạn nộp hồ sơ: 30/06/2023,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,9 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Tầng 03, tòa Ngọc Khánh Palaza, số...","<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được ...",Nhân viên,"25, 25",Hà Nội,2023-06-30


In [107]:
for ind,row in topcv_r2.iterrows():
    prev_inds = row['industry_index']
    prev_inds_ls = [str(e) for e in list(set(pd.Series(prev_inds.split(', ')).astype(int)))]
    new_inds_ls = ', '.join(prev_inds_ls)
#     print(new_inds_ls)
    topcv_r2.at[ind, 'industry_index'] = new_inds_ls

In [109]:
topcv_r2

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr,modified_deadline
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Hạn nộp hồ sơ: 30/06/2023,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Shinhan Bank, Tầng 4, Tòa Goldenpa...","<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,"33, 29, 13",Hà Nội,2023-06-30
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Hạn nộp hồ sơ: 15/06/2023,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: Tòa nhà PaxSky, 26 Ung Văn Kh...","<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,"1, 66, 4",Hồ Chí Minh,2023-06-15
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",Hạn nộp hồ sơ: 09/06/2023,CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,['- Cần Thơ: Thốt Nốt'],"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,"11, 36, 14",Cần Thơ,2023-06-09
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,Hạn nộp hồ sơ: 09/06/2023,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"['- Hà Nội: Số 19 phố nguyễn sơn, phường ngọc ...","<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,"11, 52, 62",Hà Nội,2023-06-09
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Hạn nộp hồ sơ: 31/05/2023,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Trường Đại học Thành Đô, Hoài Đức']","<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,"66, 19, 23",Hà Nội,2023-05-31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,https://www.topcv.vn/viec-lam/nhan-vien-inboun...,Nhân Viên Inbound Sales,Hạn nộp hồ sơ: 30/06/2023,Công ty TNHH Đầu tư và Du lịch 365,Trên 9 triệu,Toàn thời gian,3 năm,"['- Hà Nội: Tầng 16, Toà nhà Ladeco, 266 Đội C...","<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...",Nhân viên,25,"Hà Nội, Hồ Chí Minh",2023-06-30
30538,https://www.topcv.vn/viec-lam/dieu-duong-truon...,Điều Dưỡng Trưởng Bệnh Viện,Hạn nộp hồ sơ: 24/05/2023,Bệnh Viện Quốc Tế DoLife,Thoả thuận,Toàn thời gian,2 năm,"['- Hà Nội: 108 Nguyễn Hoàng, Mỹ Đình, Nam Từ ...","<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Trưởng/Phó phòng,8,Hà Nội,2023-05-24
30539,https://www.topcv.vn/viec-lam/nhan-vien-le-tan...,Nhân Viên Lễ Tân,Hạn nộp hồ sơ: 24/05/2023,Công ty Cổ phần Dịch vụ Y tế SBB,7 - 12 triệu,Toàn thời gian,Dưới 1 năm,"['- Hà Nội: 499 Trần Khát Chân, Hai Bà Trưng']","<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",Nhân viên,8,Hà Nội,2023-05-24
30540,https://www.topcv.vn/viec-lam/nhan-vien-dat-li...,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,Hạn nộp hồ sơ: 30/06/2023,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,9 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Tầng 03, tòa Ngọc Khánh Palaza, số...","<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được ...",Nhân viên,25,Hà Nội,2023-06-30


Check duplicates again

In [111]:
topcv_r2[topcv_r2.duplicated()]

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr,modified_deadline


So our dataset has 30,542 unique jobs that may fall within multiple categories

In [113]:
topcv_r2.to_csv('data/topcv/csv/topcv_no_dups.csv', index=False)

##### Reading the file

In [5]:
topcv_un = pd.read_csv("data/topcv/csv/topcv_no_dups.csv")

### Continue mapping industries

In [6]:
# inds_df = pd.DataFrame.from_records(industries)[['name']]
# inds_df['index'] = range(0, inds_df.shape[0])

In [7]:
# inds_df.to_csv("data/topcv/csv/industries_csv.csv", index=False)

In [7]:
inds_df = pd.read_csv("data/topcv/csv/industries_csv.csv")

In [8]:
inds_df.loc[1, 'name']

'IT phần mềm'

In [9]:
mapped_industries_ls = []
for row in topcv_un.industry_index:
    inds_ls = row.split(', ')
    mapped_ls = []
    for ind in inds_ls:
        mapped_ind = inds_df.loc[inds_df['index']==int(ind)]['name'].values[0]
        mapped_ls.append(mapped_ind)
    mapped_industries_ls.append(', '.join(mapped_ls))

In [10]:
topcv_un['mapped_industries'] = mapped_industries_ls

In [11]:
topcv_un

,url,job_title,job_deadline,company_name,job_salary,job_type,job_yoe,job_addresses,job_desc,job_req,job_level,industry_index,mod_addr,modified_deadline,mapped_industries
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Hạn nộp hồ sơ: 30/06/2023,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Shinhan Bank, Tầng 4, Tòa Goldenpa...","<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,"33, 29, 13",Hà Nội,2023-06-30,"Dịch vụ khách hàng, Xuất nhập khẩu, Nhân sự"
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Hạn nộp hồ sơ: 15/06/2023,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"['- Hồ Chí Minh: Tòa nhà PaxSky, 26 Ung Văn Kh...","<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,"1, 66, 4",Hồ Chí Minh,2023-06-15,"IT phần mềm, Hàng cao cấp, Tư vấn"
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",Hạn nộp hồ sơ: 09/06/2023,CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,['- Cần Thơ: Thốt Nốt'],"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,"11, 36, 14",Cần Thơ,2023-06-09,"Thời trang, Công nghệ Ô tô, Tổ chức sự kiện / ..."
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,Hạn nộp hồ sơ: 09/06/2023,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"['- Hà Nội: Số 19 phố nguyễn sơn, phường ngọc ...","<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,"11, 52, 62",Hà Nội,2023-06-09,"Thời trang, Bảo trì / Sửa chữa, NGO / Phi chín..."
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Hạn nộp hồ sơ: 31/05/2023,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Trường Đại học Thành Đô, Hoài Đức']","<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,"66, 19, 23",Hà Nội,2023-05-31,"Hàng cao cấp, Cơ khí / Chế tạo / Tự động hóa, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,https://www.topcv.vn/viec-lam/nhan-vien-inboun...,Nhân Viên Inbound Sales,Hạn nộp hồ sơ: 30/06/2023,Công ty TNHH Đầu tư và Du lịch 365,Trên 9 triệu,Toàn thời gian,3 năm,"['- Hà Nội: Tầng 16, Toà nhà Ladeco, 266 Đội C...","<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...",Nhân viên,25,"Hà Nội, Hồ Chí Minh",2023-06-30,Ngành nghề khác
30538,https://www.topcv.vn/viec-lam/dieu-duong-truon...,Điều Dưỡng Trưởng Bệnh Viện,Hạn nộp hồ sơ: 24/05/2023,Bệnh Viện Quốc Tế DoLife,Thoả thuận,Toàn thời gian,2 năm,"['- Hà Nội: 108 Nguyễn Hoàng, Mỹ Đình, Nam Từ ...","<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Trưởng/Phó phòng,8,Hà Nội,2023-05-24,Quản lý chất lượng (QA/QC)
30539,https://www.topcv.vn/viec-lam/nhan-vien-le-tan...,Nhân Viên Lễ Tân,Hạn nộp hồ sơ: 24/05/2023,Công ty Cổ phần Dịch vụ Y tế SBB,7 - 12 triệu,Toàn thời gian,Dưới 1 năm,"['- Hà Nội: 499 Trần Khát Chân, Hai Bà Trưng']","<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",Nhân viên,8,Hà Nội,2023-05-24,Quản lý chất lượng (QA/QC)
30540,https://www.topcv.vn/viec-lam/nhan-vien-dat-li...,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,Hạn nộp hồ sơ: 30/06/2023,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,9 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"['- Hà Nội: Tầng 03, tòa Ngọc Khánh Palaza, số...","<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được 

Ta nhận thấy TopCV không thực sự label chính xác lĩnh vực của các nghề nghiệp

Ví dụ, công việc tư vấn tài chính lại được liệt kê vào danh sách "Nhân sự"

Do đó ta sẽ mapping lại label dựa trên các keyword trong tên công việc (Job title)

In [155]:
full_inds_df = pd.read_csv('data/topcv/csv/mapping-titles - topcv_industries.csv', encoding='utf-8')
full_inds_df

,id,name,mapped_id,keywords
0,0,Kinh doanh / Bán hàng,26.0,"kinh doanh , bán hàng , sales , telesales , sale , bán , mua hàng , thu ngân , livestrea..."
1,1,IT phần mềm,21.0,"it , phần mềm , developer , web , network , devops , lập trình , thuật toán , game , ba..."
2,2,Hành chính / Văn phòng,18.0,"hành chính , văn phòng , hành chánh"
3,3,Giáo dục / Đào tạo,15.0,"giáo dục , đào tạo , giáo viên , giảng viên , trợ giảng , gia sư , học viên , training , ..."
4,4,Tư vấn,52.0,"tư vấn , consultant , consulting , cố vấn , phân tích , tư vấn , hỗ trợ , support , hướn..."
...,...,...,...,...
73,73,Thời vụ - Hợp đồng ngắn hạn,69.0,"cộng tác viên , ctv"
74,74,AI - Data Science - Business Intelligence,11.0,"business intelligence , data science , data engineer , data engineering , data analysis , d..."
75,75,Thương mại điện tử,46.0,"e-commerce , ecommerce , tmđt"
76,76,Thể Thao - Thể hình,73.0,"huấn luyện viên , gym , thể thao , thể hình , thể dục , vận động viên , yoga , runner"


In [133]:
jt = topcv_un.iloc[1]['job_title'].lower()
for e in topcv_un.iloc[1]['industry_index'].split(', '):
    row = full_inds_df.loc[full_inds_df['id']==int(e)]
    kw_ls = row['keywords'].values[0]
    for word in kw_ls.split(', '):
        if word in jt:
            print(row['name'])

1    IT phần mềm
Name: name, dtype: object


In [156]:
mapped_inds_ls = []
mapped_kw_ls = []
for index,row in topcv_un.iterrows():
    jt = ud.normalize('NFC', ' ' + row['job_title'] + ' ').lower()
    mapped_inds_index = []
    mapped_kw_index = []
    for ind_j, ind_r in full_inds_df.iterrows():
        ind_id = str(ind_r['id'])
        if (int(ind_id) == 7) | (int(ind_id) == 17):
            continue
        if int(ind_id) != 25: 
            for word in ind_r['keywords'].split(', '):
                if ud.normalize('NFC', word).lower() in jt:
                    mapped_inds_index.append(ind_id)
                    mapped_kw_index.append(ud.normalize('NFC', word).lower())
                    break
    if len(mapped_inds_index) < 1:
        mapped_inds_index.append('25')
    mapped_inds_ls.append(', '.join(mapped_inds_index))
    mapped_kw_ls.append(', '.join(mapped_kw_index))

In [157]:
topcv_un['mod_ind_index'] = mapped_inds_ls
topcv_un['mod_ind_kw'] = mapped_kw_ls

In [87]:
topcv_un[topcv_un['mod_ind_index'] == '25'][['job_title','mod_ind_index']].to_csv('data/topcv/csv/other_jobs.csv', index=False)

In [158]:
topcv_un[topcv_un['mod_ind_index'] == '25'][['job_title','mod_ind_index']]

,job_title,mod_ind_index
81,Giao Dịch Viên Tại Hà Nội,25
107,Quality Assurance (Up To 2000 Usd Net),25
154,Nhân Viên Thu Phí Tại Hải Phòng,25
218,Designer,25
265,Si Engineer (Kĩ Sư Tích Hợp Hệ Thống),25
...,...,...
30491,Phụ Tá Nha Khoa,25
30502,Nhân Viên Quầy Kính,25
30508,Trưởng Khoa Dược,25
30513,Nhân Viên Visa,25


In [160]:
pd.options.display.max_colwidth = 100
# topcv_un[['job_title','mod_ind_index', 'mod_ind_kw']].iloc[0]['mod_ind_kw']
topcv_un[['job_title','mod_ind_index', 'mod_ind_kw']].sample(20)

,job_title,mod_ind_index,mod_ind_kw
24436,Nhân Viên Hỗ Trợ Kỹ Thuật Đtdđ/Laptop [Quận 10 - Hồ Chí Minh],"4, 9","hỗ trợ , kỹ thuật"
10189,Chuyên Viên Hỗ Trợ Kinh Doanh Thái Bình,"0, 4","kinh doanh , hỗ trợ"
27381,Tổ Phó Sản Xuất Linh Kiện Nhựa,41,sản xuất
13283,Nhân Viên Sales Consultant - Mức Lương 10 - 15 Triệu/ Tháng,"0, 4","sales , consultant"
1038,IOS Developer (1200$ - 1500$),1,developer
12525,Trưởng Nhóm Tư Vấn Qua Điện Thoại - Telesales Team Leader,"0, 4, 24, 33, 69","telesales , tư vấn , leader , telesales , điện"
11419,Nhân Viên Tư Vấn Bán Hàng Tại Bình Dương,"0, 4","bán hàng , tư vấn"
19507,QA/QC (Khách Hàng Nước Ngoài),8,qa/qc
17160,Chuyên Viên Tư Vấn Tuyển Sinh ( Telesale),"3, 4","tuyển sinh , tư vấn"
24724,Product Manager - Cosmetic,"24, 34","manager , product manager"


In [162]:
topcv_un.to_csv("data/topcv/csv/topcv_mod_ind.csv", index=False)

### Mapping salaries

In [3]:
topcv_mi = pd.read_csv('data/topcv/csv/topcv_mod_ind.csv')

In [4]:
topcv_mi = topcv_mi.drop(['job_deadline', 'job_addresses', 'industry_index'], axis=1)

In [5]:
reg = '[\p{L}-]+'
currencies_sr = topcv_mi['job_salary'].apply(lambda x: re.findall(reg, x)[-1])
range_sr = topcv_mi['job_salary'].apply(lambda x: re.findall(reg, x)[0])
check_others = topcv_mi['job_salary'].apply(lambda x: len(re.findall(reg, x)) > 2)

In [6]:
currencies_sr.value_counts()

triệu    21833
thuận     8101
USD        608
Name: job_salary, dtype: int64

In [7]:
range_sr.value_counts()

-       14352
Thoả     8101
Trên     4922
Tới      3167
Name: job_salary, dtype: int64

In [8]:
check_others.sum()

0

Vậy ta chắc chắn rằng:
- Về tiền tệ: Chỉ có đơn vị triệu (1,000,000 VND) và USD (23,000 VND)
- Về từ ngữ chỉ giới hạn: Trên (min) và Tới (max)

Từ đây ta có thể dễ dàng mapping lương 

In [88]:
def convert_currency(index, value):
    crc = currencies_sr.iloc[index]
    if float(value) == 0:
        return np.nan
    if crc == 'triệu':
        return float(value) * 1000000
    elif crc == 'USD':
        if float(value) < 1000000:
            return float(value) * 23000
        else:
            return float(value)
    return np.nan

In [11]:
topcv_mi.iloc[33]

url                  https://www.topcv.vn/viec-lam/nhan-vien-stack-...
job_title                                    Nhân Viên Stack Developer
company_name         CÔNG TY TNHH TRUYỀN THÔNG VÀ DỊCH VỤ DU LỊCH C...
job_salary                                               Tới 2,000 USD
job_type                                                Toàn thời gian
job_yoe                                                          2 năm
job_desc             <div class="content-tab"><p>- Tham gia xây dựn...
job_req              <div class="content-tab"><p>- Thành thạo về lậ...
job_level                                                    Nhân viên
mod_addr                                                        Hà Nội
modified_deadline                                           2023-06-14
mapped_industries                 IT phần mềm, Hàng cao cấp, Kiến trúc
mod_ind_index                                                        1
mod_ind_kw                                                  developer 
Name: 

In [17]:
num_reg = '\d+'
topcv_mi['job_salary'].apply(lambda x: re.findall(num_reg, x.replace(',','')))

0        [10, 30]
1              []
2         [7, 10]
3            [10]
4        [15, 30]
           ...   
30537         [9]
30538          []
30539     [7, 12]
30540     [9, 12]
30541     [7, 15]
Name: job_salary, Length: 30542, dtype: object

In [89]:
min_salary_ls = []
max_salary_ls = []
num_reg = '\d+'
salaries_raw = topcv_mi['job_salary'].apply(lambda x: re.findall(num_reg, x.replace(',','')))
for index,item in enumerate(salaries_raw):
    if len(item) < 1:
        min_salary_ls.append(np.nan)
        max_salary_ls.append(np.nan)
    elif len(item) == 1:
        range_kw = range_sr.iloc[index]
        if range_kw == 'Trên':
            min_salary_ls.append(convert_currency(index, item[0]))
            max_salary_ls.append(np.nan)
        else:
            max_salary_ls.append(convert_currency(index, item[0]))
            min_salary_ls.append(np.nan)
    else:
        min_salary_ls.append(convert_currency(index, item[0]))
        max_salary_ls.append(convert_currency(index, item[-1]))

In [90]:
topcv_mi['min_salary'] = min_salary_ls
topcv_mi['max_salary'] = max_salary_ls

In [33]:
topcv_mi.iloc[33]

url                  https://www.topcv.vn/viec-lam/nhan-vien-stack-...
job_title                                    Nhân Viên Stack Developer
company_name         CÔNG TY TNHH TRUYỀN THÔNG VÀ DỊCH VỤ DU LỊCH C...
job_salary                                               Tới 2,000 USD
job_type                                                Toàn thời gian
job_yoe                                                          2 năm
job_desc             <div class="content-tab"><p>- Tham gia xây dựn...
job_req              <div class="content-tab"><p>- Thành thạo về lậ...
job_level                                                    Nhân viên
mod_addr                                                        Hà Nội
modified_deadline                                           2023-06-14
mapped_industries                 IT phần mềm, Hàng cao cấp, Kiến trúc
mod_ind_index                                                        1
mod_ind_kw                                                  developer 
min_sa

In [40]:
topcv_mi.iloc[11325]

url                  https://www.topcv.vn/viec-lam/english-native-t...
job_title                                       English Native Teacher
company_name                             Công ty TNHH phát triển AROMA
job_salary                                                 23 - 26 USD
job_type                                                 Bán thời gian
job_yoe                                                     Dưới 1 năm
job_desc             <div class="content-tab"><p>- Class type: Corp...
job_req              <div class="content-tab"><p>- Native English s...
job_level                                                    Nhân viên
mod_addr                                                   Hồ Chí Minh
modified_deadline                                           2023-06-15
mapped_industries    Kinh doanh / Bán hàng, Ngành nghề khác, Báo ch...
mod_ind_index                                                        3
mod_ind_kw                                                    teacher 
min_sa

### Mapping job levels

In [41]:
for e in topcv_mi['job_level'].value_counts().index:
    print(e)

Nhân viên
Thực tập sinh
Trưởng nhóm
Trưởng/Phó phòng
Quản lý / Giám sát
Giám đốc
Trưởng chi nhánh
Phó giám đốc


In [42]:
topcv_mi['job_level'].value_counts()

Nhân viên             24883
Thực tập sinh          1494
Trưởng nhóm            1482
Trưởng/Phó phòng       1206
Quản lý / Giám sát     1127
Giám đốc                241
Trưởng chi nhánh         72
Phó giám đốc             37
Name: job_level, dtype: int64

In [43]:
mapped_levels = pd.read_csv('data/topcv/csv/mapping-levels - topcv-levels.csv').set_index('name')
mapped_levels

,id,mapped_id
name,,
Nhân viên,1,6
Thực tập sinh,2,9
Trưởng nhóm,3,4
Trưởng/Phó phòng,4,3
Quản lý / Giám sát,5,4
Giám đốc,6,2
Trưởng chi nhánh,7,2
Phó giám đốc,8,2


In [44]:
mapped_lv_inds = []
for e in topcv_mi['job_level']:
    mapped_lv_inds.append(mapped_levels['mapped_id'].loc[e])

In [45]:
pd.Series(mapped_lv_inds).value_counts()

6    24883
4     2609
9     1494
3     1206
2      350
dtype: int64

In [46]:
pd.Series(mapped_lv_inds)

0        6
1        6
2        6
3        6
4        9
        ..
30537    6
30538    3
30539    6
30540    6
30541    6
Length: 30542, dtype: int64

In [47]:
topcv_mi['mod_level'] = mapped_lv_inds

In [48]:
topcv_mi.head(5)

,url,job_title,company_name,job_salary,job_type,job_yoe,job_desc,job_req,job_level,mod_addr,modified_deadline,mapped_industries,mod_ind_index,mod_ind_kw,min_salary,max_salary,mod_level
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,Hà Nội,2023-06-30,"Dịch vụ khách hàng, Xuất nhập khẩu, Nhân sự","4, 20, 33","tư vấn , tài chính , khách hàng",10000000.0,30000000.0,6
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,Hồ Chí Minh,2023-06-15,"IT phần mềm, Hàng cao cấp, Tư vấn",1,game,NaN,NaN,6
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,Cần Thơ,2023-06-09,"Thời trang, Công nghệ Ô tô, Tổ chức sự kiện / ...","19, 52","vận hành , vận hành",7000000.0,10000000.0,6
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,Hà Nội,2023-06-09,"Thời trang, Bảo trì / Sửa chữa, NGO / Phi chín...","0, 47, 55","kinh doanh , dụng cụ, dụng cụ",10000000.0,NaN,6
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,Hà Nội,2023-05-31,"Hàng cao cấp, Cơ khí / Chế tạo / Tự động hóa, ...","3, 9","giảng viên , công nghệ thông tin",15000000.0,30000000.0,9


### Mapping cities

In [49]:
mapped_cities = pd.read_csv('data/topcv/csv/mapping-cities - cities.csv')

In [50]:
mapped_ct_df = mapped_cities[['name', 'id']].set_index('name')

In [51]:
mapped_cities_set = set(mapped_cities.name)

In [52]:
topcv_cities_set = set(topcv_mi['mod_addr'].str.split(', ').explode().unique())

In [53]:
same_cities = topcv_cities_set & mapped_cities_set

In [54]:
not_in_other = topcv_cities_set ^ mapped_cities_set

In [55]:
not_in_map = topcv_cities_set - mapped_cities_set
not_in_topcv = mapped_cities_set - topcv_cities_set

In [56]:
print(same_cities)

{'Hà Nội', 'Cần Thơ', 'Quảng Ninh', 'Hà Giang', 'Hưng Yên', 'Sơn La', 'Bắc Giang', 'Bạc Liêu', 'Tuyên Quang', 'Vĩnh Long', 'Đồng Nai', 'Đồng Tháp', 'Thừa Thiên Huế', 'Quảng Ngãi', 'Quảng Nam', 'Hải Dương', 'Bắc Ninh', 'Miền Nam', 'Bình Dương', 'Miền Trung', 'Long An', 'Bình Thuận', 'Miền Bắc', 'Ninh Thuận', 'Trà Vinh', 'Thái Nguyên', 'Bến Tre', 'Lào Cai', 'Lạng Sơn', 'Tiền Giang', 'Bình Phước', 'Bình Định', 'Lai Châu', 'Yên Bái', 'Ninh Bình', 'Sóc Trăng', 'Cao Bằng', 'Gia Lai', 'Hà Nam', 'Bắc Kạn', 'Đà Nẵng', 'Quảng Bình', 'Nghệ An', 'Nam Định', 'Cà Mau', 'Tây Ninh', 'Lâm Đồng', 'Hà Tĩnh', 'Đắk Lắk', 'Thái Bình', 'Hải Phòng', 'Hậu Giang', 'Vĩnh Phúc', 'An Giang', 'Điện Biên', 'Phú Thọ', 'Quảng Trị', 'Phú Yên', 'Kiên Giang', 'Kon Tum'}


In [57]:
not_in_map

{'Bà RịaVũng Tàu',
 'Cửu Long',
 'Hoà Bình',
 'Hồ Chí Minh',
 'Khánh Hoà',
 'Nước Ngoài',
 'Thanh Hoá',
 'Toàn Quốc',
 'Đắc Nông'}

In [58]:
not_in_topcv

{'Bà Rịa - Vũng Tàu',
 'Hòa Bình',
 'Khác',
 'Khánh Hòa',
 'Quốc tế',
 'TP.HCM',
 'Thanh Hóa',
 'Toàn quốc',
 'ĐBSCL',
 'Đắk Nông'}

Mapping cities

In [59]:
same_ct_dict = {}
for ct in same_cities:
    same_ct_dict[ct] = ct
same_ct_dict['Bà RịaVũng Tàu'] = 'Bà Rịa - Vũng Tàu'
same_ct_dict['Cửu Long'] = 'ĐBSCL'
same_ct_dict['Hoà Bình'] = 'Hòa Bình'
same_ct_dict['Hồ Chí Minh'] = 'TP.HCM'
same_ct_dict['Khánh Hoà'] = 'Khánh Hòa'
same_ct_dict['Nước Ngoài'] = 'Quốc tế'
same_ct_dict['Thanh Hoá'] = 'Thanh Hóa'
same_ct_dict['Toàn Quốc'] = 'Toàn quốc'
same_ct_dict['Đắc Nông'] = 'Đắk Nông'

In [60]:
mapped_ct_dict = {}
for key,value in same_ct_dict.items():
    mapped_ct_id = mapped_ct_df['id'].loc[value]
    mapped_ct_dict[key] = mapped_ct_id

In [61]:
mapping_topcv_cities_df = pd.DataFrame({
    'name': mapped_ct_dict.keys(),
    'mapped_id': mapped_ct_dict.values()
}).sort_values('name')
mapping_topcv_cities_df['id'] = range(1, mapping_topcv_cities_df.shape[0]+1)

In [62]:
mapping_topcv_cities_df[['id', 'name', 'mapped_id']].to_csv('data/topcv/csv/mapping-cities-topcv-cities.csv', index=False)

In [63]:
len(same_ct_dict)

69

In [64]:
mapped_ct_df = mapping_topcv_cities_df[['name', 'mapped_id']].set_index('name')

In [65]:
mapped_ct_ind_ls = []
for ct_ls in topcv_mi['mod_addr'].str.split(', '):
    ct_ind_ls = []
    for ct in ct_ls:
        ct_ind_ls.append(mapped_ct_df['mapped_id'].loc[ct])
    mapped_ct_ind_ls.append(', '.join([str(e) for e in ct_ind_ls]))

In [66]:
topcv_mi['mod_city'] = mapped_ct_ind_ls

In [67]:
topcv_mi

,url,job_title,company_name,job_salary,job_type,job_yoe,job_desc,job_req,job_level,mod_addr,modified_deadline,mapped_industries,mod_ind_index,mod_ind_kw,min_salary,max_salary,mod_level,mod_city
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,Hà Nội,2023-06-30,"Dịch vụ khách hàng, Xuất nhập khẩu, Nhân sự","4, 20, 33","tư vấn , tài chính , khách hàng",10000000.0,30000000.0,6,25
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,Hồ Chí Minh,2023-06-15,"IT phần mềm, Hàng cao cấp, Tư vấn",1,game,NaN,NaN,6,66
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,Cần Thơ,2023-06-09,"Thời trang, Công nghệ Ô tô, Tổ chức sự kiện / ...","19, 52","vận hành , vận hành",7000000.0,10000000.0,6,13
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,Hà Nội,2023-06-09,"Thời trang, Bảo trì / Sửa chữa, NGO / Phi chín...","0, 47, 55","kinh doanh , dụng cụ, dụng cụ",10000000.0,NaN,6,25
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,Hà Nội,2023-05-31,"Hàng cao cấp, Cơ khí / Chế tạo / Tự động hóa, ...","3, 9","giảng viên , công nghệ thông tin",15000000.0,30000000.0,9,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,https://www.topcv.vn/viec-lam/nhan-vien-inboun...,Nhân Viên Inbound Sales,Công ty TNHH Đầu tư và Du lịch 365,Trên 9 triệu,Toàn thời gian,3 năm,"<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...",Nhân viên,"Hà Nội, Hồ Chí Minh",2023-06-30,Ngành nghề khác,0,sales,9000000.0,NaN,6,"25, 66"
30538,https://www.topcv.vn/viec-lam/dieu-duong-truon...,Điều Dưỡng Trưởng Bệnh Viện,Bệnh Viện Quốc Tế DoLife,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Trưởng/Phó phòng,Hà Nội,2023-05-24,Quản lý chất lượng (QA/QC),16,điều dưỡng,NaN,NaN,3,25
30539,https://www.topcv.vn/viec-lam/nhan-vien-le-tan...,Nhân Viên Lễ Tân,Công ty Cổ phần Dịch vụ Y tế SBB,7 - 12 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",Nhân viên,Hà Nội,2023-05-24,Quản lý chất lượng (QA/QC),33,lễ tân,7000000.0,12000000.0,6,25
30540,https://www.topcv.vn/viec-lam/nhan-vien-dat-li...,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,9 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được ...",Nhân viên,Hà Nội,2023-06-30,Ngành nghề khác,"0, 14, 31","bán hàng , sự kiện , du lịch",9000000.0,12000000.0,6,25


### Mapping years of experience

In [68]:
topcv_mi['job_yoe'].value_counts().sort_index()

1 năm                        9856
2 năm                        5310
3 năm                        2526
4 năm                         300
5 năm                         507
Dưới 1 năm                   4880
Không yêu cầu kinh nghiệm    6823
Trên 5 năm                    340
Name: job_yoe, dtype: int64

In [69]:
num_reg = '\d+'
min_yoe_ls = []
max_yoe_ls = []
for yoe in topcv_mi['job_yoe']:
    if yoe == 'Không yêu cầu kinh nghiệm':
        min_yoe_ls.append(0)
        max_yoe_ls.append(0)
    elif 'Dưới' in yoe:
        min_yoe_ls.append(0)
        max_yoe_ls.append(int(re.findall(num_reg, yoe)[0]))
    elif 'Trên' in yoe:
        min_yoe_ls.append(int(re.findall(num_reg, yoe)[0]) + 1)
        max_yoe_ls.append(np.nan)
    else:
        min_yoe_ls.append(int(re.findall(num_reg, yoe)[0]))
        max_yoe_ls.append(np.nan)

In [70]:
topcv_mi['mod_min_year'] = min_yoe_ls
topcv_mi['mod_max_year'] = max_yoe_ls

In [71]:
topcv_mi

,url,job_title,company_name,job_salary,job_type,job_yoe,job_desc,job_req,job_level,mod_addr,modified_deadline,mapped_industries,mod_ind_index,mod_ind_kw,min_salary,max_salary,mod_level,mod_city,mod_min_year,mod_max_year
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,Hà Nội,2023-06-30,"Dịch vụ khách hàng, Xuất nhập khẩu, Nhân sự","4, 20, 33","tư vấn , tài chính , khách hàng",10000000.0,30000000.0,6,25,0,0.0
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,Hồ Chí Minh,2023-06-15,"IT phần mềm, Hàng cao cấp, Tư vấn",1,game,NaN,NaN,6,66,2,NaN
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,Cần Thơ,2023-06-09,"Thời trang, Công nghệ Ô tô, Tổ chức sự kiện / ...","19, 52","vận hành , vận hành",7000000.0,10000000.0,6,13,0,1.0
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,Hà Nội,2023-06-09,"Thời trang, Bảo trì / Sửa chữa, NGO / Phi chín...","0, 47, 55","kinh doanh , dụng cụ, dụng cụ",10000000.0,NaN,6,25,2,NaN
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,Hà Nội,2023-05-31,"Hàng cao cấp, Cơ khí / Chế tạo / Tự động hóa, ...","3, 9","giảng viên , công nghệ thông tin",15000000.0,30000000.0,9,25,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,https://www.topcv.vn/viec-lam/nhan-vien-inboun...,Nhân Viên Inbound Sales,Công ty TNHH Đầu tư và Du lịch 365,Trên 9 triệu,Toàn thời gian,3 năm,"<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...",Nhân viên,"Hà Nội, Hồ Chí Minh",2023-06-30,Ngành nghề khác,0,sales,9000000.0,NaN,6,"25, 66",3,NaN
30538,https://www.topcv.vn/viec-lam/dieu-duong-truon...,Điều Dưỡng Trưởng Bệnh Viện,Bệnh Viện Quốc Tế DoLife,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Trưởng/Phó phòng,Hà Nội,2023-05-24,Quản lý chất lượng (QA/QC),16,điều dưỡng,NaN,NaN,3,25,2,NaN
30539,https://www.topcv.vn/viec-lam/nhan-vien-le-tan...,Nhân Viên Lễ Tân,Công ty Cổ phần Dịch vụ Y tế SBB,7 - 12 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",Nhân viên,Hà Nội,2023-05-24,Quản lý chất lượng (QA/QC),33,lễ tân,7000000.0,12000000.0,6,25,0,1.0
30540,https://www.topcv.vn/viec-lam/nhan-vien-dat-li...,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,9 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được ...",Nhân viên,Hà Nội,2023-06-30,Ngành nghề khác,"0, 14, 31","bán hàng , sự kiện , du lịch",9000000.0,12000000.0,6,25,0,0.0


# Mapping correct industries

In [72]:
mapped_ind_df = pd.read_csv("data/topcv/csv/mapping-titles - industries.csv").set_index('id')
mapped_ind_df

,name,nameEn
id,,
1,An ninh - Bảo vệ,Safety - Security
2,An toàn lao động,HSE
3,Biên phiên dịch,Interpreter - Translator
4,Bác sĩ,Medical Doctor
5,Bán buôn - Bán lẻ - Quản lý cửa hàng,Retail - Business Management
...,...,...
70,Spa - Làm đẹp,Spa - Beauty
71,Bưu chính - Viễn thông,Posts and Telecommunications
72,Nghiên cứu và Phát triển,Research and Development


In [73]:
full_inds_df = pd.read_csv("data/topcv/csv/mapping-titles - topcv_industries.csv")
full_inds_df = full_inds_df.set_index('id').copy()

In [74]:
full_inds_df['mapped_id'].loc[0]

26.0

In [75]:
true_inds_ls = []
true_text_inds_ls = []
for inds_ls in topcv_mi['mod_ind_index'].str.split(', '):
    temp_inds_ls = []
    temp_text_inds_ls = []
    for ind in inds_ls:
        true_ind = int(full_inds_df['mapped_id'].loc[int(ind)])
        temp_inds_ls.append(true_ind)
        temp_text_inds_ls.append(mapped_ind_df['name'].loc[true_ind])
    true_inds_ls.append(', '.join([str(e) for e in temp_inds_ls]))
    true_text_inds_ls.append(', '.join([str(e) for e in temp_text_inds_ls]))

In [76]:
topcv_mi['true_inds'] = true_inds_ls
topcv_mi['true_text_inds'] = true_text_inds_ls

In [85]:
topcv_mi

,url,job_title,company_name,job_salary,job_type,job_yoe,job_desc,job_req,job_level,mod_addr,...,mod_ind_index,mod_ind_kw,min_salary,max_salary,mod_level,mod_city,mod_min_year,mod_max_year,true_inds,true_text_inds
0,https://www.topcv.vn/viec-lam/chuyen-vien-cao-...,Chuyên Viên Cao Cấp Tư Vấn Tài Chính - Mảng Tí...,Ngân Hàng TNHH MTV Shinhan Việt Nam,10 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p><strong>Làm việc t...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Nhân viên,Hà Nội,...,"4, 20, 33","tư vấn , tài chính , khách hàng",10000000.0,30000000.0,6,25,0,0.0,"52, 33, 8","Tư vấn, Ngân hàng, Chăm sóc khách hàng"
1,https://www.topcv.vn/viec-lam/senior-game-desi...,Senior Game Designer,Công ty TNHH Gameloft,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p><strong>Main respo...","<div class=""content-tab""><ul><li>Above 2 years...",Nhân viên,Hồ Chí Minh,...,1,game,NaN,NaN,6,66,2,NaN,21,IT Phần mềm
2,https://www.topcv.vn/viec-lam/nhan-vien-van-ha...,"Nhân Viên Vận Hành Máy (Tách Màu, Máy Xát Trắn...",CHI NHÁNH NHÀ MÁY GẠO HOA SEN - CÔNG TY TNHH X...,7 - 10 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Nhận nhiệm vụ hà...","<div class=""content-tab""><p>- Tốt nghiệp từ Tr...",Nhân viên,Cần Thơ,...,"19, 52","vận hành , vận hành",7000000.0,10000000.0,6,13,0,1.0,"9, 55","Cơ khí, Vận hành máy - Bảo trì - Bảo dưỡng thi..."
3,https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...,Nhân Viên Kinh Doanh Dụng Cụ Cắt Gọt,CÔNG TY TNHH MỘT THÀNH VIÊN MACHINERY & TOOLS ...,Trên 10 triệu,Toàn thời gian,2 năm,"<div class=""content-tab""><p>Tìm kiến nguồn khá...","<div class=""content-tab""><p>Tốt nghiệp chuyên ...",Nhân viên,Hà Nội,...,"0, 47, 55","kinh doanh , dụng cụ, dụng cụ",10000000.0,NaN,6,25,2,NaN,"26, 65, 64","Kinh doanh, Hàng tiêu dùng, Hàng gia dụng"
4,https://www.topcv.vn/viec-lam/giang-vien-nganh...,Giảng Viên Ngành Công Nghệ Thông Tin,Trường Đại học Thành Đô,15 - 30 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><p>- Quản lý, tổ chức...","<div class=""content-tab""><p>- Thạc sĩ trở lên,...",Thực tập sinh,Hà Nội,...,"3, 9","giảng viên , công nghệ thông tin",15000000.0,30000000.0,9,25,0,0.0,"15, 21","Giáo dục - Đào tạo, IT Phần mềm"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30537,https://www.topcv.vn/viec-lam/nhan-vien-inboun...,Nhân Viên Inbound Sales,Công ty TNHH Đầu tư và Du lịch 365,Trên 9 triệu,Toàn thời gian,3 năm,"<div class=""content-tab""><p>- Giao dịch với kh...","<div class=""content-tab""><p>- Có kinh nghiệm t...",Nhân viên,"Hà Nội, Hồ Chí Minh",...,0,sales,9000000.0,NaN,6,"25, 66",3,NaN,26,Kinh doanh
30538,https://www.topcv.vn/viec-lam/dieu-duong-truon...,Điều Dưỡng Trưởng Bệnh Viện,Bệnh Viện Quốc Tế DoLife,Thoả thuận,Toàn thời gian,2 năm,"<div class=""content-tab""><p>- Tổ chức, chỉ đạo...","<div class=""content-tab""><p>- Tốt nghiệp Đại h...",Trưởng/Phó phòng,Hà Nội,...,16,điều dưỡng,NaN,NaN,3,25,2,NaN,58,Y tế - Chăm sóc sức khỏe
30539,https://www.topcv.vn/viec-lam/nhan-vien-le-tan...,Nhân Viên Lễ Tân,Công ty Cổ phần Dịch vụ Y tế SBB,7 - 12 triệu,Toàn thời gian,Dưới 1 năm,"<div class=""content-tab""><p>- Đón tiếp khách h...","<div class=""content-tab""><p>- Nam/Nữ, độ tuổi ...",Nhân viên,Hà Nội,...,33,lễ tân,7000000.0,12000000.0,6,25,0,1.0,8,Chăm sóc khách hàng
30540,https://www.topcv.vn/viec-lam/nhan-vien-dat-li...,Nhân Viên Đặt Lịch Hẹn Sự Kiện Mảng Du Lịch - ...,CÔNG TY CỔ PHẦN CÂU LẠC BỘ KỲ NGHỈ VIỆT NAM,9 - 12 triệu,Toàn thời gian,Không yêu cầu kinh nghiệm,"<div class=""content-tab""><ul><li>Tiếp cận với ...","<div class=""content-tab""><ul><li>Sử dụng được ...",Nhân viên,Hà Nội,...,"0, 14, 31","bán hàng , sự kiện , du lịch",9000000.0,12000000.0,6,25,0,0.0,"26, 32, 25","Kinh doanh, Nghề nghiệp khác, Khách sạn - Nhà ..."


In [91]:
topcv_mi.iloc[12943]

url                  https://www.topcv.vn/viec-lam/nhan-vien-kinh-d...
job_title            Nhân Viên Kinh Doanh Tại Hà Nội, Hồ Chí Minh (...
company_name                                 Công ty CP Kỹ thuật Temas
job_salary                                         Trên 23,000,000 USD
job_type                                                Toàn thời gian
job_yoe                                                     Dưới 1 năm
job_desc             <div class="content-tab"><p>- Tìm kiếm và phát...
job_req              <div class="content-tab"><p>- Tốt nghiệp Cao đ...
job_level                                                    Nhân viên
mod_addr                                           Hà Nội, Hồ Chí Minh
modified_deadline                                           2023-05-31
mapped_industries    Kinh doanh / Bán hàng, Thời trang, Bảo trì / S...
mod_ind_index                                                        0
mod_ind_kw                                                 kinh doanh 
min_sa

In [92]:
topcv_mi.to_csv('data/topcv/csv/topcv_done_mappings_raw.csv', index=False)

In [79]:
# other_jobs = topcv_un[topcv_un['mod_ind_index'] == '25'][['job_title','mod_ind_index']]
# filter_ws = ['giao dịch viên', 'lễ tân', 'quầy kính', 'visa', 'vệ sinh siêu thị', 'thu phí', 'tiếng trung', 'tiếng nhật', 'hồ sơ',
#              'văn thư', 'thư viện', 'nhân viên admin', 'receptionist', 'sổ đỏ',
#              'vận hành', 'đo lường', 'seo', 'điều khiển', 'account', '3d', 'helpdesk', 'system admin', 
#              'máy tính', 'business analyst', 'insurance', 'phát sóng', 'kỹ thuật', 'điều dưỡng', 'ads', 'ba', 'leader',
#             'performance executive', 'dev', 'kỹ sư giải pháp', 'bridge system engineer', 'dinh dưỡng', 'sale', 'bồi thường',
#             'dms', 'relationship manager','brse', 'hệ thống', 'system engineer', 'php', 'bác sĩ', 'product manager',
#             'chất lượng', 'tài sản', 'phân tích nghiệp vụ', 'mua hàng', 'thu ngân', 'nghiên cứu', 'phát triển', 'tester',
#             'học viên', 'r&d', 'financial', 'finance', 'operations manager', 'bệnh viện', 'consulting', 'consultant', 
#             'trình dược viên', 'nhập khẩu', 'cộng tác viên', 'animator', 'animation', 'training', 'thu mua', 'kế hoạch',
#             'trưởng phòng', 'giao nhận', 'software', 'embedded', 'kỹ thuật', 'technical', 'mobile', '.net', 'tuyển dụng',
#             'dữ liệu', 'hàng nhập', 'restaurant', 'tmđt', 'kĩ thuật', 'livestream', 'kol', 'koc', 'livestream/tiktok/kol/koc',
#             'điều phối', 'hr', 'thợ hàn', 'c&b', 'giao hàng', 'đơn hàng', 'assistant', 'nợ', 'quản trị', 'bưu cục', 'java', 'mua hàng',
#             'giám đốc', 'tài xế', 'chứng từ', 'cố vấn', 'hải quan', 'scrum', 'maketing', 'pricing', 'triển khai', 'front end', 'back end',
#             'front-end', 'back-end', 'frontend', 'backend', 'fullstack', 'full-stack', 'full stack', 'xnk', 'thanh toán', 'operation',
#             'supply chain', 'supply chains', 'planning', 'reactjs', 'react native', 'javascript', 'pháp chế', 'computer vision', 'computer',
#             'fresher', 'content', 'business intelligence', 'systems administration', 'front - end', 'kiểm thử', 'biển', 'ai',
#             'artificial intelligence', 'hydraulic', 'thủy lực', 'điều độ', 'branding', 'cửa hàng', 'bếp phụ', 'human resources',
#             'e-commerce','ecommerce', 'garment', 'merchandiser', 'planner', 'flutter', 'bảo vệ', 'modelling', 'phân tích', 'lead',
#             'cứu hộ', 'nlp', 'marketer', 'maketer', 'tuyển sinh', 'supply', 'merchandise', 'phục trang', 'video', 'videographer',
#             'truyền thông', 'brand', 'biên tập viên', 'pccc', 'seeding', 'merchandising', 'biên kịch', 'legal', 'bia rượu',
#             'social media', 'bảo hành', 'lái xe', 'lái phụ', 'cntt', 'nodejs', 'full - stack', 'sáng tạo nội dung', 'học sinh',
#             'server', 'y học', 'research', 'stylist', 'tín hiệu', 'pr', 'mechanical', '2d', 'human resource', 'analyst', 'event',
#             'crm', 'tổng đài', 'testing', 'bếp chính', 'tutor', 'teacher', 'phần cứng', 'may mẫu', 'academic', 'academy', 'học vụ',
#              'chủ nhiệm', 'may mặc', 'bác sỹ', 'dealer', 'nhập khẩu', 'xuất khẩu', 'thị trường', 'manager', 'giáo vụ', 'học vụ',
#             'bếp phó', 'fontend', 'purchasing', 'cung ứng', 'python', 'ios', 'hộ sinh', 'cskh', 'quảng cáo', 'bếp', 'năng lượng',
#             'photographer', 'fraud', 'risk', 'executive', 'c & b', 'robotics', 'dựng phim',
#             'huấn luyện viên', 'gym', 'thể thao', 'thể hình', 'thể dục', 'vận động viên', 'trắc địa', 'ad optimizer', 'vi sinh', 'đầu tư',
#             'kết cấu', 'test', 'marketor', 'tiệc', 'callcenter', 'call center', 'call-center', 'giảng dạy', 'industrial', 'buồng phòng',
#             'ios', 'android', 'nextjs', 'nuxtjs', 'golang', 'education', 'tech', 'học thuật', 'client service', 'camera', 'camercaman',
#             'quay phim', 'learning experience', 'recruit', 'recruitment', 'recruiter', 'talent acquisition', 'talent management', 'creative',
#             'mep', 'hóa sinh', 'kỹ sư hóa', 'trắc đạc', 'construction', 'kỹ sư qs', 'business administration', 'business administrator',
#             'investment', 'asset', 'investor', 'invest', 'angular', 'thi công', 'accounting', 'accountant', 'audit', 'auditing', 'audits',
#             'auditor', 'electrical', 'electric', 'electrician', 'electronic', 'electronics', 'c++', 'csharp', 'c-sharp', 'c#', 'cloud',
#             'platform engineer', 'warehouse staff', 'data warehouse', 'data science', 'data engineer', 'data engineering', 'data analysis', 'data analyst', 
#              'data analytics', 'ai', 'artificial intelligence', 'phân tích', 'modelling', 'modeling', 'nlp', 'analyst', 'data warehouse', 'chef',
#             'công trường', 'nguồn vốn', 'thủy sản', 'hải sản', 'quan trắc', 'dịch thuật', 'runner', 'dịch vụ hành khách', 'cây xanh', 'google ads',
#             'google adwords', 'nhiễm khuẩn', 'phóng viên', 'cms', 'business', 'doanh nghiệp',
#             'quan hệ công chúng', 'đối ngoại', 'đối nội', 'quan hệ quốc tế', 'business collaborator', 'internal communications', 
#             'internal communicator', 'internal communication', 'external relations', 'internal relations', 'hợp tác quốc tế', 
#             'quan hệ doanh nghiệp', 'hợp tác doanh nghiệp', 'y tế', 'y tá', 'đấu thầu', 'qs', 'quan hệ đối tác', 'dotnet', 'tuân thủ', 
#             'compliance', 'sinh viên', 'cashier', 'giám sát hạ tầng', 'giám sát', 'housekeeping', 'tạp vụ', 'chăm sóc cư dân',
#             'chăm sóc căn hộ', 'chăm sóc dân cư', 'art', 'service engineer', 'btv', 'rủi ro', 'thẩm định', 'ứng viên', 'import', 'export',
#             'đóng hàng', 'set up vision', 'computer vision', 'machine learning', 'học liệu', 'sommelier', 'hóa lý', 'cầu đường', 'treasury',
#             'đóng gói', 'dược sỹ', 'trưởng phòng', 'phó phòng', 'director', 'administrative', 'trồng mới', 'matlab', 'quản đốc',
#             'administrator', 'cad engineer', 'học đường', 'khảo thí', 'giám thị', 'thí sinh', 'rnd', 'quay dựng', 'pme', 'seller', 'youtube',
#             'kỹ sư cầu', 'gatehouse', 'fanpage', 'adoption specialist', 'thoát nước', 'bđs', 'quay - dựng', 'điều vận', 'translator',
#             'tiền lương và phúc lợi', 'hành chánh', 'sql', 'admin', 'hỗ trợ', 'support', 'supervisor', 'giám sát', 'tiktok', 'lắp ráp',
#             'tổ trưởng', 'chỉ huy', 'mua sắm', 'ctv', 'trực page', 'multimedia', 'hướng dẫn viên', 'xe cơ giới', 'hàn xì', 'tham vấn',
#             'vmware', 'bim', 'service desk', 'structural engineer', 'plumbing', 'plumber', 'firefighting', 'firefighter']
# def check_other_jobs(title):
#     title_lw = ud.normalize('NFC',title).lower()
#     for w in filter_ws:
#         if ud.normalize('NFC',w) in title_lw:
#             return False
#     return True
# print(other_jobs[other_jobs['job_title'].apply(check_other_jobs)].shape)
# other_jobs[other_jobs['job_title'].apply(check_other_jobs)].sample(20)